In [1]:
from keras.layers                   import Input, Dense, SimpleRNN, GRU, LSTM, CuDNNLSTM, RepeatVector
from keras.models                 import Model
from keras.layers.core           import Flatten
from keras.callbacks              import LambdaCallback 
from keras.optimizers           import SGD, RMSprop, Adam
from keras.layers.wrappers      import Bidirectional as Bi
from keras.layers.wrappers      import TimeDistributed as TD
from keras.layers                     import merge
from keras.applications.vgg16   import VGG16 
from keras.layers.normalization import BatchNormalization as BN
from keras.layers.noise           import GaussianNoise as GN
from keras.layers.merge         import Concatenate
from keras.layers.core            import Dropout
from keras.layers.merge         import Concatenate as Concat
from keras.layers.noise           import GaussianNoise as GN
from keras.layers.merge         import Dot,Multiply
from keras import backend as K
import tensorflow as tf
from tensorflow.python import debug as tf_debug
from tensorflow.python.debug.lib.debug_data import has_inf_or_nan
import pandas as pd

input   = Input(shape=(227,))
e = Dense(227, activation='linear')(input)
e = Dense(227, activation='relu')(e)
e = Dense(128, activation='relu')(e)
e = Dense(96, activation='linear')(e)

d = Dense(96, activation='relu')(e)
d = Dense(128, activation='relu')(d)
d = Dense(227, activation='relu')(d)
d = Dense(227, activation='linear')(d)

dae = Model(input, d)
dae.compile(optimizer=Adam(), loss='mse')

def set_debugger_session():
    sess = K.get_session()
    sess = tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.add_tensor_filter('has_inf_or_nan', has_inf_or_nan)
    K.set_session(sess)
#set_debugger_session()

/home/gimpei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tdf = pd.read_csv('vars/one_hot_train.csv')
Tdf = pd.read_csv('vars/one_hot_test.csv')
df = pd.concat([tdf, Tdf], axis=0)

In [3]:
print(tdf.shape)
print(Tdf.shape)

(595212, 229)
(892816, 229)


In [4]:
for c in tdf.columns:
    print(c)

id
ps_calc_01
ps_calc_02
ps_calc_03
ps_calc_04
ps_calc_05
ps_calc_06
ps_calc_07
ps_calc_08
ps_calc_09
ps_calc_10
ps_calc_11
ps_calc_12
ps_calc_13
ps_calc_14
ps_calc_15_bin
ps_calc_16_bin
ps_calc_17_bin
ps_calc_18_bin
ps_calc_19_bin
ps_calc_20_bin
ps_car_11
ps_car_12
ps_car_13
ps_car_14
ps_car_15
ps_ind_01
ps_ind_03
ps_ind_06_bin
ps_ind_07_bin
ps_ind_08_bin
ps_ind_09_bin
ps_ind_10_bin
ps_ind_11_bin
ps_ind_12_bin
ps_ind_13_bin
ps_ind_14
ps_ind_15
ps_ind_16_bin
ps_ind_17_bin
ps_ind_18_bin
ps_reg_01
ps_reg_02
ps_reg_03
target
ps_car_01_cat_-1
ps_car_01_cat_0
ps_car_01_cat_1
ps_car_01_cat_10
ps_car_01_cat_11
ps_car_01_cat_2
ps_car_01_cat_3
ps_car_01_cat_4
ps_car_01_cat_5
ps_car_01_cat_6
ps_car_01_cat_7
ps_car_01_cat_8
ps_car_01_cat_9
ps_car_02_cat_-1
ps_car_02_cat_0
ps_car_02_cat_1
ps_car_03_cat_-1
ps_car_03_cat_0
ps_car_03_cat_1
ps_car_04_cat_0
ps_car_04_cat_1
ps_car_04_cat_2
ps_car_04_cat_3
ps_car_04_cat_4
ps_car_04_cat_5
ps_car_04_cat_6
ps_car_04_cat_7
ps_car_04_cat_8
ps_car_04_cat_9
ps_

In [6]:
Tdf['target'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: target, dtype: float64

In [7]:
df = df.set_index('id')
df = df.drop(['target'], axis=1)

In [8]:
df.head()

,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,...,ps_ind_04_cat_0,ps_ind_04_cat_1,ps_ind_05_cat_-1,ps_ind_05_cat_0,ps_ind_05_cat_1,ps_ind_05_cat_2,ps_ind_05_cat_3,ps_ind_05_cat_4,ps_ind_05_cat_5,ps_ind_05_cat_6
id,,,,,,,,,,,,,,,,,,,,,
7,0.148880,0.050045,-0.251084,0.180445,-0.241681,0.456249,-0.392864,0.160989,-0.321177,-0.354606,...,0,1,0,1,0,0,0,0,0,0
9,-0.148498,-0.359428,-0.148623,-0.106058,-0.241681,0.302359,0.381890,-0.271185,-0.321177,-0.152857,...,1,0,0,1,0,0,0,0,0,0
13,0.049037,0.252580,-0.359341,-0.106058,0.050391,0.302359,-0.392864,-0.271185,-0.077570,-0.152857,...,0,1,0,1,0,0,0,0,0,0
16,0.148880,0.477143,-0.359341,-0.106058,0.451634,-0.186895,-0.392864,-0.271185,0.372501,-0.470425,...,1,0,0,1,0,0,0,0,0,0
17,-0.049345,0.150009,-0.476988,-0.106058,0.050391,-0.365047,0.011013,0.160989,-0.077570,0.354226,...,0,1,0,1,0,0,0,0,0,0


In [9]:
from sklearn.cross_validation import KFold
import swap_noise

#for k in range(100):
NFOLDS=100
SEED=777
kf = KFold(len(df.values), n_folds=NFOLDS, shuffle=True, random_state=SEED)
for i, (train_index, test_index) in enumerate(kf):

    noised = swap_noise.noise(df.values)
    dae.fit(noised[train_index], df.values[train_index],
                    epochs=1,
                    validation_data=(noised[test_index], df.values[test_index]),
                    batch_size=6000,
                    shuffle=True,)
    lr = 0.001*( (NFOLDS - i)/NFOLDS )
    print('now epoch', i, lr)
    K.set_value(dae.optimizer.lr, lr)

dae.save_weights('vars/dae.h5')

/home/gimpei/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


now noising
start rand
finish rand
finish noising
Train on 1473147 samples, validate on 14881 samples
Epoch 1/1
1473147/1473147 [==============================] - 6s 4us/step - loss: 0.0309 - val_loss: 0.0211
now epoch 0 0.001
now noising
start rand
finish rand
finish noising
Train on 1473147 samples, validate on 14881 samples
Epoch 1/1
1473147/1473147 [==============================] - 4s 3us/step - loss: 0.0191 - val_loss: 0.0178
now epoch 1 0.00099
now noising
start rand
finish rand
finish noising
Train on 1473147 samples, validate on 14881 samples
Epoch 1/1
1473147/1473147 [==============================] - 4s 3us/step - loss: 0.0171 - val_loss: 0.0165
now epoch 2 0.00098
now noising
start rand
finish rand
finish noising
Train on 1473147 samples, validate on 14881 samples
Epoch 1/1
1473147/1473147 [==============================] - 4s 3us/step - loss: 0.0161 - val_loss: 0.0159
now epoch 3 0.0009699999999999999
now noising
start rand
finish rand
finish noising
Train on 1473147 sampl

1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0136 - val_loss: 0.0135
now epoch 35 0.0006500000000000001
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0135 - val_loss: 0.0135
now epoch 36 0.00064
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0135 - val_loss: 0.0135
now epoch 37 0.00063
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0134 - val_loss: 0.0134
now epoch 38 0.00062
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - l

1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0130 - val_loss: 0.0129
now epoch 70 0.0003
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0129 - val_loss: 0.0130
now epoch 71 0.00029
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0129 - val_loss: 0.0129
now epoch 72 0.00028000000000000003
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - loss: 0.0129 - val_loss: 0.0130
now epoch 73 0.00027
now noising
start rand
finish rand
finish noising
Train on 1473148 samples, validate on 14880 samples
Epoch 1/1
1473148/1473148 [==============================] - 4s 3us/step - l